## Datenquelle
http://www.offenedaten.frankfurt.de/dataset/frankfurter-stadtteilgrenzen-fur-gis-systeme

- Importiert in QGIS
- Exportiert als csv-Datei

In [1]:
%matplotlib notebook

In [2]:
#!pip uninstall tensorflow protobuf --yes

In [3]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
#!pip install geopy 
from geopy.geocoders import Nominatim
#!pip install folium tensorflow-tensorboard tensorflow
import folium
from folium import plugins

In [5]:
# The code was removed by Watson Studio for sharing.

The geograpical coordinate of Altstadt are 50.11104545, 8.68293572698541.
The geograpical coordinate of Innenstadt are 50.11287765, 8.67492196213115.
The geograpical coordinate of Bahnhofsviertel are 50.10774145, 8.66873601221096.
The geograpical coordinate of Westend-Süd are 50.1152448, 8.6622696.
The geograpical coordinate of Westend-Nord are 50.1263563, 8.6679209.
The geograpical coordinate of Nordend-West are 50.1249136, 8.67795.
The geograpical coordinate of Nordend-Ost are 50.1249197, 8.6923167.
The geograpical coordinate of Ostend are 50.1160632, 8.72011225465195.
The geograpical coordinate of Bornheim are 50.13305575, 8.71493241229322.
The geograpical coordinate of Gutleutviertel are 50.0978128, 8.65076020625548.
The geograpical coordinate of Gallus are 50.10365845, 8.63670648468984.
The geograpical coordinate of Bockenheim are 50.12236065, 8.63753199693013.
The geograpical coordinate of Sachsenhausen-N. are 50.1073316, 8.6876717.
The geograpical coordinate of Sachsenhausen-S. 

,Stadtteil,Latitude,Longitude
0,Altstadt,50.111045,8.682936
1,Innenstadt,50.112878,8.674922
2,Bahnhofsviertel,50.107741,8.668736
3,Westend-Süd,50.115245,8.662270
4,Westend-Nord,50.126356,8.667921
5,Nordend-West,50.124914,8.677950
6,Nordend-Ost,50.124920,8.692317
7,Ostend,50.116063,8.720112
8,Bornheim,50.133056,8.714932
9,Gutleutviertel,50.097813,8.650760


In [6]:
# create map of New York using latitude and longitude values
map_ffm = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df_ffm['Latitude'], df_ffm['Longitude'], df_ffm['Stadtteil']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        tooltip=borough,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ffm)  
    
map_ffm

In [7]:
# The code was removed by Watson Studio for sharing.

df_ffm[df_ffm['Stadtteil']=='Innenstadt']['Stadtteil']

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    lat_list=[]
    lng_list=[]
    popup_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        lat_list = [(v['venue']['location']['lat']) for v in results]
        lng_list = [(v['venue']['location']['lng']) for v in results]
        marker_list= list(zip(lat_list,lng_list))
        
        popup_list = [(v['venue']['name']) for v in results]
        
        folium.plugins.MarkerCluster(
            locations=marker_list,
            popups=popup_list
            ).add_to(map_ffm)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
LIMIT = 100
borough_venues = getNearbyVenues(names=df_ffm['Stadtteil'],
                                   latitudes=df_ffm['Latitude'],
                                   longitudes=df_ffm['Longitude']
                                  )
map_ffm

Altstadt
Innenstadt
Bahnhofsviertel
Westend-Süd
Westend-Nord
Nordend-West
Nordend-Ost
Ostend
Bornheim
Gutleutviertel
Gallus
Bockenheim
Sachsenhausen-N.
Sachsenhausen-S.
Flughafen
Oberrad
Niederrad
Schwanheim
Griesheim
Rödelheim
Hausen
Praunheim
Heddernheim
Niederursel
Ginnheim
Dornbusch
Eschersheim
Eckenheim
Preungesheim
Bonames
Berkersheim
Riederwald
Seckbach
Fechenheim
Höchst
Nied
Sindlingen
Zeilsheim
Unterliederbach
Sossenheim
Nieder-Erlenbach
Kalbach-Riedberg
Harheim
Nieder-Eschbach
Bergen-Enkheim
Frankfurter Berg


In [11]:
print(borough_venues.shape)
borough_venues.head()

(847, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt,50.111045,8.682936,Kleinmarkthalle,50.112778,8.682958,Market
1,Altstadt,50.111045,8.682936,Weinterasse Rollanderhof,50.112473,8.682164,Wine Bar
2,Altstadt,50.111045,8.682936,Römerberg,50.110489,8.682131,Plaza
3,Altstadt,50.111045,8.682936,SCHIRN Kunsthalle,50.110291,8.683542,Art Museum
4,Altstadt,50.111045,8.682936,Dom Aussichtsplattform,50.110609,8.684908,Scenic Lookout


In [12]:
borough_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Altstadt,100,100,100,100,100,100
Bahnhofsviertel,100,100,100,100,100,100
Bergen-Enkheim,1,1,1,1,1,1
Berkersheim,2,2,2,2,2,2
Bockenheim,22,22,22,22,22,22
Bonames,9,9,9,9,9,9
Bornheim,18,18,18,18,18,18
Dornbusch,4,4,4,4,4,4
Eckenheim,7,7,7,7,7,7


In [13]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 183 uniques categories.


In [14]:
# one hot encoding
borough_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
borough_onehot['Neighbourhood'] = borough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [borough_onehot.columns[-1]] + list(borough_onehot.columns[:-1])
borough_onehot = borough_onehot[fixed_columns]

borough_onehot.head()

,Neighbourhood,ATM,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Apple Wine Pub,Argentinian Restaurant,Art Gallery,Art Museum,...,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Well,Whisky Bar,Wine Bar
0,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Altstadt,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
borough_onehot.shape

(847, 184)

In [16]:
borough_grouped = borough_onehot.groupby('Neighbourhood').mean().reset_index()
borough_grouped

,Neighbourhood,ATM,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Apple Wine Pub,Argentinian Restaurant,Art Gallery,Art Museum,...,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Well,Whisky Bar,Wine Bar
0,Altstadt,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.00,0.00,0.020000
1,Bahnhofsviertel,0.000000,0.01,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,...,0.000000,0.010000,0.000000,0.010000,0.010000,0.03,0.000000,0.00,0.01,0.000000
2,Bergen-Enkheim,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000
3,Berkersheim,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000
4,Bockenheim,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.045455,0.000000,0.00,0.000000,0.00,0.00,0.045455
5,Bonames,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000
6,Bornheim,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.055556
7,Dornbusch,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000
8,Eckenheim,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000
9,Eschersheim,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000


In [17]:
borough_grouped.shape

(46, 184)

In [18]:
num_top_venues = 10

for hood in borough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = borough_grouped[borough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Altstadt----
                venue  freq
0                Café  0.10
1          Restaurant  0.06
2               Plaza  0.05
3   German Restaurant  0.05
4                 Bar  0.03
5          Art Museum  0.03
6      Clothing Store  0.03
7            Wine Bar  0.02
8           Nightclub  0.02
9  Italian Restaurant  0.02


----Bahnhofsviertel----
                venue  freq
0               Hotel  0.10
1   Indian Restaurant  0.05
2                 Bar  0.05
3  Seafood Restaurant  0.04
4    Asian Restaurant  0.03
5        Burger Joint  0.03
6              Bakery  0.03
7  Chinese Restaurant  0.03
8          Restaurant  0.03
9                Café  0.03


----Bergen-Enkheim----
                 venue  freq
0                Plaza   1.0
1      Other Nightlife   0.0
2  Moroccan Restaurant   0.0
3      Motorcycle Shop   0.0
4               Museum   0.0
5          Music Store   0.0
6          Music Venue   0.0
7            Nightclub   0.0
8         Noodle House   0.0
9          Opera House   0